In [7]:
from astropy.io import fits
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.cluster import *
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import astropy
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import constants as const
import hdbscan
from astropy.io import fits

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchvision import datasets, transforms
from torch.autograd import Variable
# from torchvision.utils import save_image

In [632]:
# открыть файлы с скрасными смещениями

data = fits.open('http://gal-03.sai.msu.ru/~vtoptun/photometry/rcsed_v2_clean.fits')

data.info()

Filename: /root/.astropy/cache/download/py3/0508754bf44ea7119c5b6b1a661568ef
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (73531,)   uint8   
  1  Joined        1 BinTableHDU   1644   4105516R x 559C   [J, D, D, D, D, D, K, I, J, I, E, J, 6A, D, I, 15A, J, J, D, D, D, D, E, E, E, J, J, J, 40A, B, B, E, I, I, 10A, E, J, E, J, E, I, 14A, E, I, 33A, L, L, E, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, E, E, E, E, E, D, D, D, D, D, E, E, E, E, E, D, D, D, D, D, E, E, E, E, E, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, E, E, E, E, E, E, E, E, E, E, E, D, E, D, E, D, E, D, E, D, E, D, E, D, E, D, E, D, E, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, D, D, D, D, D, D, D, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E

In [633]:
GrID = pd.read_csv('rcsed_iGrID.csv')

In [ ]:
data = data[1].data

In [ ]:
cols = data.columns.names #559

In [ ]:
# конвертация в датафрейм
RCSED_V2 = pd.DataFrame(np.array(data).byteswap().newbyteorder())
display(RCSED_V2.head())

In [ ]:
ID_cols = list(RCSED_V2.filter(regex='ID'))

In [ ]:
#ID_cols.append('ind')
ID_cols.append('recno_uzc')
ID_cols.append('ind')

In [ ]:
ID_cols

In [ ]:
RCSED_V2 = RCSED_V2[RCSED_V2.columns.drop(ID_cols)]

In [ ]:
cat_cols = ['SpecFile_wigglez', 'targetname_6df', 'SeqNum_2df', 'planid_lamost']

In [ ]:
RCSED_V2 = RCSED_V2[RCSED_V2.columns.drop(cat_cols)]

In [ ]:
len(RCSED_V2.columns)

In [ ]:
RCSED_V2.head()

In [ ]:
len_data = len(RCSED_V2)

In [ ]:
len_data

In [ ]:
sum(np.array(RCSED_V2.OBJNO_deep2) != -2147483648) / len_data

In [ ]:
#replace NaN

In [ ]:
def null_treating(cols, null):
    for col in cols:
        if sum(np.array(RCSED_V2[col]) == null) / len(RCSED_V2) > 0.9:
            RCSED_V2.drop([col],axis=1, inplace=True)
            cols.remove(col)
        else:
            RCSED_V2[col].replace(null, np.nan) 

In [ ]:
#null = -2147483648
cols_NaN1 = ['mjd_sdss', 'q_z_2df', 'specid_6df', 'obsid_lamost', 'lmjd_lamost', 'mjd_lamost',
             'OBJNO_deep2', 'OBJNO_deep3']

In [ ]:
#null = -32768
cols_NaN2 = ['plate_sdss', 'quality_6df', 'f_z_lega_c', 'Q_wigglez', 'NQ_gama']

In [ ]:
#null = 255
cols_NaN3 = ['spid_lamost', 'fiberid_lamost']

In [ ]:
null_treating(cols_NaN1, -2147483648)

In [ ]:
null_treating(cols_NaN2, -32768)

In [ ]:
null_treating(cols_NaN3, 255)

In [ ]:
len(RCSED_V2.columns)

In [ ]:
#  Anomaly detection

top_values = []
nan = []

for col in RCSED_V2.columns:
    vc = RCSED_V2[col].value_counts(ascending=False).head(1)
    nan = (RCSED_V2[col].isna()).sum()
    total = nan
    if vc[vc.index[0]] >1000:
        total = vc[vc.index[0]]+nan
    top_values.append([vc.name, vc.index[0], vc[vc.index[0]], nan, total])
    
anomaly = pd.DataFrame(data=top_values, columns = ['column_name', 'anomaly_value', 'value_counts', 'nan', 'total'])

In [ ]:
anomaly.sort_values(by='total', ascending=False)[1:20]

In [ ]:
#4105516*0.9 = 3694964

col_anomaly = anomaly[anomaly.total>3694964].column_name.to_list()

In [ ]:
len(col_anomaly)

In [ ]:
RCSED_V2.drop(col_anomaly, axis=1, inplace=True)

In [ ]:
len(RCSED_V2.columns)

In [ ]:
sns.distplot(RCSED_V2.w3sigmag_7_wise)
plt.title('w3sigmag_7_wise')
plt.show()

In [ ]:
len(np.unique(RCSED_V2.w3sigmag_7_wise))-2202889

**To do**
*IterativeImputer*
-> models each feature with missing values as a function of other features, and uses that estimate for imputation

In [ ]:
len(np.unique(RCSED_V2.KAPERMAG3ERR_ukidss))-2967341

In [ ]:
col_for_nan = anomaly2.sort_values(by='total', ascending=False)[1:149].column_name.to_list()
col_anomaly = anomaly2.sort_values(by='total', ascending=False)[1:149].anomaly_value.to_list()

In [ ]:
for col, anomaly in zip(col_for_nan, col_anomaly):
    RCSED_V2[col].replace(anomaly, np.nan)

In [ ]:
data_0 = RCSED_V2.fillna(0)

In [ ]:
data_0

In [ ]:
data_0 = data_0.replace([np.inf, -np.inf], np.nan).fillna(0)

# Labeled part of data

In [ ]:
GrID.head()

In [ ]:
len(GrID)

In [ ]:
sdss_indx = GrID[-GrID.iGrID.isna()].index.to_list()

In [ ]:
len(sdss_indx)

In [ ]:
data_0.ind

In [ ]:
x = data_0[data_0.ind.isin(sdss_indx)]

In [ ]:
len(x)

In [ ]:
y = GrID.iloc[sdss_indx]

In [ ]:
s = pd.Series(y.iGrID).value_counts()

In [ ]:
one_sized = y[y.iGrID.isin(s[s == 1].index)].index.to_list()

In [ ]:
y_without_one = y[~y.iGrID.isin(s[s == 1].index)]

In [ ]:
len(y_without_one)

In [ ]:
x_without_one = x[~x.ind.isin(one_sized)]

In [ ]:
x_without_one.head()

# VAE

In [438]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x_without_one)
display(x)

array([[-2.96068453e+00, -8.47137893e-01,  1.36122092e+00, ...,
         5.88998002e-05,  3.56226173e-01,  2.54646558e-04],
       [-2.96064404e+00, -1.35645424e-01,  1.95708374e+00, ...,
         3.61292801e-03,  4.12418385e-01,  2.47764867e-03],
       [-2.96063036e+00, -1.40620777e+00,  2.54383877e+00, ...,
         8.25179404e-04,  3.93781511e-01,  9.52398655e-04],
       ...,
       [ 2.80389843e+00, -1.01314244e-01, -5.85270884e-02, ...,
        -9.70410412e-04, -4.84952697e-01, -6.00629403e-04],
       [ 2.80390023e+00, -8.60379393e-01,  3.69015423e-02, ...,
        -9.70410412e-04, -4.84952697e-01, -6.00629403e-04],
       [ 2.80390203e+00, -4.82020008e-01, -4.40446706e-01, ...,
        -3.39085502e-04,  2.85324190e-01,  2.75540075e-05]])

In [439]:
x.shape

(220777, 347)

In [440]:
Y = np.array(y_without_one).reshape(-1)

In [441]:
Y.shape

(220777,)

-----

In [456]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)

        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, x.shape[1]))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [457]:
vae = VAE(x_dim=x.shape[1], h_dim1= int(x.shape[1]/2), h_dim2=int(x.shape[1]/4), z_dim=1)


optimizer = optim.Adam(vae.parameters())
train_loader = torch.utils.data.DataLoader(dataset=x, batch_size=int(len(x)/1000), shuffle=True)#, num_workers=20)

def loss_function(recon_x, x, mu, log_var):
    criterion = nn.MSELoss().cuda()
    BCE = criterion(recon_x, x.view(-1, x.shape[1]))
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx in range(400):
        #data = torch.tensor(x.sample(n=1000).astype(np.float32).values)
        data = torch.tensor(x[np.random.choice(range(x.shape[0]), 1000)].astype(np.float32))
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [459]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(1, 25)):
    train(epoch)

Train Epoch: 1 [0/220777 (0%)]	Loss: 0.002165
Train Epoch: 1 [100000/220777 (10%)]	Loss: 0.001440
Train Epoch: 1 [200000/220777 (20%)]	Loss: 0.002923
Train Epoch: 1 [300000/220777 (30%)]	Loss: 0.001182
====> Epoch: 1 Average loss: 89975084.6788
Train Epoch: 2 [0/220777 (0%)]	Loss: 0.001076
Train Epoch: 2 [100000/220777 (10%)]	Loss: 0.013517
Train Epoch: 2 [200000/220777 (20%)]	Loss: 0.004049
Train Epoch: 2 [300000/220777 (30%)]	Loss: 0.001038
====> Epoch: 2 Average loss: 0.0885
Train Epoch: 3 [0/220777 (0%)]	Loss: 0.001133
Train Epoch: 3 [100000/220777 (10%)]	Loss: 0.000955
Train Epoch: 3 [200000/220777 (20%)]	Loss: 0.000953
Train Epoch: 3 [300000/220777 (30%)]	Loss: 0.001516
====> Epoch: 3 Average loss: 0.0384
Train Epoch: 4 [0/220777 (0%)]	Loss: 0.001392
Train Epoch: 4 [100000/220777 (10%)]	Loss: 0.000907
Train Epoch: 4 [200000/220777 (20%)]	Loss: 0.000791
Train Epoch: 4 [300000/220777 (30%)]	Loss: 0.000656
====> Epoch: 4 Average loss: 0.0041
Train Epoch: 5 [0/220777 (0%)]	Loss: 0.00

KeyboardInterrupt: 

In [460]:
mu, log_var = vae.encoder(torch.tensor(x.astype(np.float32)))
embed = vae.sampling(mu, log_var).detach().numpy()

In [543]:
hdbScan = hdbscan.hdbscan_.HDBSCAN(min_cluster_size=2, min_samples=10, 
                                   allow_single_cluster=False, core_dist_n_jobs=20).fit(embed)

In [544]:
m2 = homogeneity_completeness_v_measure(Y,hdbScan.labels_)
m2

(0.5330250424733153, 0.7729184646698886, 0.6309383142618596)

In [546]:
pd.Series(hdbScan.labels_).value_counts(ascending=True).tail(15)

 1005      150
 1234      155
 3281      158
 1302      166
 230       168
 1595      179
 2321      193
 1780      199
 1788      203
 5106      209
 1348      210
 70        231
 1211      244
 912       485
-1       52273
dtype: int64

In [547]:
pd.Series(Y).value_counts(ascending=True).tail(15)

191838.0    166
101773.0    176
176164.0    178
20078.0     181
20266.0     208
13370.0     219
160410.0    221
127065.0    223
30885.0     230
112271.0    239
42643.0     266
176804.0    389
39456.0     547
153084.0    739
149473.0    861
dtype: int64

In [465]:
z_dim = 1
input_size = x.shape[1]
n_samples = x.shape[0]

h_dim1 = int(x.shape[1]/2)
h_dim2 = int(x.shape[1]/4)

In [479]:
n_samples

220777

In [485]:
class AVAE(nn.Module):
    def __init__(self):
        super(AVAE, self).__init__()
        
        self.gen_l1 = torch.nn.Linear(z_dim, h_dim2)
        self.gen_l2 = torch.nn.Linear(h_dim2, h_dim1)
        self.gen_l3 = torch.nn.Linear(h_dim1, input_size)
        
        self.enc_l1 = torch.nn.Linear(input_size+z_dim, h_dim1)
        self.enc_l2 = torch.nn.Linear(h_dim1, h_dim2)
        self.enc_l3 = torch.nn.Linear(h_dim2, z_dim)
        
        self.disc_l1 = torch.nn.Linear(input_size+z_dim, h_dim1)
        self.disc_l2 = torch.nn.Linear(h_dim1, h_dim2)
        self.disc_l3 = torch.nn.Linear(h_dim2, 1)
    
    def sample_prior(self, s):
        if self.training:
            m = torch.zeros((s.data.shape[0], z_dim))
            std = torch.ones((s.data.shape[0], z_dim))
            d = Variable(torch.normal(m,std))
        else:
            d = Variable(torch.zeros((s.data.shape[0], z_dim)))
            
        return d
        
    def discriminator(self, x,z):
        i = torch.cat((x, z), dim=1)
        h = F.relu(self.disc_l1(i))
        h = F.relu(self.disc_l2(h))
        return self.disc_l3(h)  
        
    def sample_posterior(self, x):
        i = torch.cat((x, self.sample_prior(x)), dim=1)
        h = F.relu(self.enc_l1(i))
        h = F.relu(self.enc_l2(h))
        return self.enc_l3(h)
    
    def decoder(self, z):
        i = F.relu(self.gen_l1(z))
        h = F.relu(self.gen_l2(i))
        return torch.sigmoid(self.gen_l3(h))
        
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, x.shape[1]))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

    def forward(self, x):
        z_p = self.sample_prior(x)
        
        z_q = self.sample_posterior(x)
        
        log_d_prior = self.discriminator(x, z_p)
        log_d_posterior = self.discriminator(x, z_q)
        
        disc_loss = torch.mean(
            nn.functional.binary_cross_entropy_with_logits(
            log_d_posterior, torch.ones_like(log_d_posterior)
        )
        + nn.functional.binary_cross_entropy_with_logits(
            log_d_prior, torch.zeros_like(log_d_prior))
        )
        
        x_recon = self.decoder(z_q)
        recon_liklihood = -nn.functional.binary_cross_entropy(
                                                x_recon, x)*x.data.shape[0]
        
        gen_loss = torch.mean(log_d_posterior)-torch.mean(recon_liklihood)
        
        return disc_loss, gen_loss

In [486]:
model = AVAE()

In [487]:
disc_params = []
gen_params = []
for name, param in model.named_parameters():
    
    if 'disc' in name:
        
        disc_params.append(param)
    else:
        gen_params.append(param)

In [488]:
disc_optimizer = torch.optim.Adam(disc_params, lr=1e-3)
gen_optimizer = torch.optim.Adam(gen_params, lr=1e-3)

In [489]:
x.shape

(220777, 347)

In [490]:
def train_AVAE(epoch, batches_per_epoch = 501, log_interval=500):
    model.train()
    
    ind = np.arange(x.shape[0])
    for i in range(batches_per_epoch):
        data = torch.from_numpy(x[np.random.choice(ind, size=batch_size)])
        data = Variable(data, requires_grad=False).float()
        
        
        discrim_loss, gen_loss= model(data)
        
        gen_optimizer.zero_grad()
        gen_loss.backward(retain_graph=True)
        gen_optimizer.step()
        
        disc_optimizer.zero_grad()
        discrim_loss.backward(retain_graph=True)
        disc_optimizer.step()
        if (i % log_interval == 0) and (epoch % 1 ==0):
            #Print progress
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}\tLoss: {:.6f}'.format(
                epoch, i * batch_size, batch_size*batches_per_epoch,
                discrim_loss.item() / len(data), gen_loss.item() / len(data)))

    print('====> Epoch: {} done!'.format(
          epoch))

In [510]:
%%time
for epoch in range(1, 15):
    train_AVAE(epoch)

Train Epoch: 1 [0/128256]	Loss: 0.001176	Loss: -4.678825
Train Epoch: 1 [128000/128256]	Loss: 0.001314	Loss: -4.578405
====> Epoch: 1 done!
Train Epoch: 2 [0/128256]	Loss: 0.001075	Loss: -4.614848
Train Epoch: 2 [128000/128256]	Loss: 0.000906	Loss: -4.888037
====> Epoch: 2 done!
Train Epoch: 3 [0/128256]	Loss: 0.001257	Loss: -5.129023
Train Epoch: 3 [128000/128256]	Loss: 0.001005	Loss: -4.264121
====> Epoch: 3 done!
Train Epoch: 4 [0/128256]	Loss: 0.001019	Loss: -4.223755
Train Epoch: 4 [128000/128256]	Loss: 0.000969	Loss: -4.254047
====> Epoch: 4 done!
Train Epoch: 5 [0/128256]	Loss: 0.001105	Loss: -4.278278
Train Epoch: 5 [128000/128256]	Loss: 0.001202	Loss: -4.698373
====> Epoch: 5 done!
Train Epoch: 6 [0/128256]	Loss: 0.001199	Loss: -5.099048
Train Epoch: 6 [128000/128256]	Loss: 0.000863	Loss: -4.109143
====> Epoch: 6 done!
Train Epoch: 7 [0/128256]	Loss: 0.001410	Loss: -4.537691
Train Epoch: 7 [128000/128256]	Loss: 0.000979	Loss: -5.109467
====> Epoch: 7 done!
Train Epoch: 8 [0/12

In [511]:
data = Variable(torch.from_numpy(x), requires_grad=False)

model.train()
zs = model.sample_posterior(data.float()).detach().numpy()



In [620]:
#import matplotlib.pyplot as plt
#%matplotlib inline

#plt.scatter(zs[:,0], zs[:, 1], c=Y)

In [ ]:
#check the shape

In [512]:
zs.shape

(220777, 1)

In [513]:
x.shape

(220777, 347)

In [ ]:
embed.shape 

# TODO: 
1) grid search for min_samples & min_cluster_size

2) optimize AVAE + graphics 

In [612]:
hdbScan2 = hdbscan.hdbscan_.HDBSCAN(min_cluster_size=2, min_samples=9, 
                                   allow_single_cluster=False, core_dist_n_jobs=20).fit(zs)

In [613]:
len(hdbScan2.labels_)

220777

In [614]:
m4 = homogeneity_completeness_v_measure(Y,hdbScan2.labels_) 
m4

(0.8438774646309881, 0.8285269211292502, 0.8361317436550977)

In [615]:
len(np.unique(hdbScan2.labels_))

72113

In [616]:
len(np.unique(Y)) #true number of clusters

56546

In [574]:
#9
pd.Series(hdbScan2.labels_).value_counts(ascending=True).tail(15)

 2737      190
 7213      202
 8573      207
 6756      214
 5223      217
 5605      218
 3819      229
 5064      237
 8695      260
 8878      273
 5456      299
 505       338
 501       759
 3142      826
-1       51137
dtype: int64

In [617]:
ls = pd.DataFrame(hdbScan2.labels_)

In [618]:
ind = ls[ls[0] == -1].index.to_list()

In [575]:
pd.Series(Y).value_counts(ascending=True).tail(15)

191838.0    166
101773.0    176
176164.0    178
20078.0     181
20266.0     208
13370.0     219
160410.0    221
127065.0    223
30885.0     230
112271.0    239
42643.0     266
176804.0    389
39456.0     547
153084.0    739
149473.0    861
dtype: int64